In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')
np.random.seed(1)

### Introduction to Polynomial (Secretly Linear) Regression and Regularization
Let's return to our example data from earlier, except this time instead of using KNN Regression, we will briefly introduce Linear Regression (which we will study in more detail next week), how to extend linear models to be (apparently) non-linear by adding features, and the concept of *Regularization* which will help us control the complexity of the linear regression model. Ultimately, the purpose of briefly introducing linear regression now is because it provides us a nice test-case where we have multiple model hyper-parameters that we will need to *tune* (or find the best settings of), and thus will motivate some of our interest in optimization which we will build upon in the future.

In [ ]:
n_samples = 30

# True Function we want to estimate
true_fun = lambda X: np.cos(1.5 * np.pi * X)

# Noisy Samples from the true function
X = np.sort(np.random.rand(n_samples))
y = true_fun(X) + np.random.randn(n_samples) * 0.4

plt.figure(figsize=(10,10))
# Plot the true function:
X_plot = np.linspace(0, 1, 100)
plt.plot(X_plot, true_fun(X_plot), '--',label="True function")
# Plot the data samples
plt.scatter(X,y, label="Samples")
plt.legend(loc="best")
plt.show()

In [ ]:
X

In [ ]:
# Just for the purposes of display, set the print precision to 2 decimals
%precision 2
t=X[:,np.newaxis]
t

In [ ]:
t2 = np.sqrt(t)
t2

In [ ]:
t3 = np.exp(t)
t3

In [ ]:
np.ones(len(t))

In [ ]:
np.hstack([t,t2,t3])

In [ ]:
from sklearn.linear_model import LinearRegression, SGDRegressor
LinearRegression

In [ ]:
lr = LinearRegression()
X_stacked_example = np.hstack([t,t2,t3])
lr.fit(X_stacked_example,y)

In [ ]:
y.shape

In [ ]:
lr.predict(X_stacked_example)

In [ ]:
# Now let's see how it does:
X_plot = np.linspace(0, 1, 100)
#plt.plot(X_plot, lr.predict(X_plot[:, np.newaxis]), label="Model")
plt.plot(X_stacked_example[:,0], lr.predict(X_stacked_example), label="Model")
plt.plot(X_plot, true_fun(X_plot), '--',label="True function")
plt.scatter(X, y, label="Samples")
plt.xlabel("x")
plt.ylabel("y")
plt.xlim((0, 1))
plt.ylim((-2, 2))
#plt.legend(loc="best")

In [ ]:
# Let's build some Polynomial features:
from sklearn.preprocessing import PolynomialFeatures
d=3
pfeatures = PolynomialFeatures(degree=d,include_bias=True)
X_new = pfeatures.fit_transform(X[:, np.newaxis])
lr = LinearRegression()
lr.fit(X_new,y)

In [ ]:
from sklearn.pipeline import Pipeline
# Here is a list of different degree polynomials to try out
degrees = [1,2,3,5,10,15,20,30]

# Generate samples of the true function + noise
X = np.sort(np.random.rand(n_samples))
noise_amount = 0.4
y = true_fun(X) + np.random.randn(n_samples) * noise_amount

# For each of the different polynomial degrees we listed above
for d in degrees:
    plt.figure(figsize=(7, 7)) # Make a new figure
    # Construct the polynomial features
    polynomial_features = PolynomialFeatures(degree=d,
                                             include_bias=False)
    # Construct linear regression model
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    # Now fit the data first through the 
    # polynomial basis, then do regression
    pipeline.fit(X[:, np.newaxis], y)
    
    # Get the accuracy score of the trained model
    # on the original training data
    score = pipeline.score(X[:, np.newaxis],y)

    # Plot the results
    X_plot = np.linspace(0, 1, 100)
    plt.plot(X_plot, pipeline.predict(X_plot[:, np.newaxis]), label="Model")
    plt.plot(X_plot, true_fun(X_plot), '--',label="True function")
    plt.scatter(X, y, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    
    # Print the polynomial degree and the training
    # accuracy in the title of the graph
    plt.title("Degree {}\nTrain score = {:.3f}".format(
        d, score))
    plt.show()   

In [ ]:
linear_regression.coef_


What is going on here? To understand this, we need to understand something about how the model is optimizing error. 

Show Linear Regression MSE Example on board and introduce the idea of loss functions.

### How do we deal with this tradeoff? Regularization!
$$
Loss = Error(w,D) + \alpha\cdot\Omega(w)
$$
where $\Omega(w)$ represents what we call a "Regularization" of the function or a "Penalty Term" The purpose of $\Omega(w)$ is to help us prevent the (otherwise complex) model from being overly complicated, by penalizing this complexity. There are many ways to do this that we will see later on, but one common way to do this for linear models is to penalize the **total weight** that you allow all of the $w_i$ to have. Specifically how one calculates this total weight turns out to matter a lot, and we shall see it return in future weeks. But to get us started in un-packing how to do this, we first need to talk about what a [Norm](https://en.wikipedia.org/wiki/Norm_(mathematics)) is, how it relates to Linear Regression weights, and how it helps us perform Regularization.

### Norms and their relationship to Regularization
A Norm is a concept in mathematics that allows us to essentially measure length or size, typically of vectors. Any time you have tried to compute the distance between two points in space (say, by using the Pythagorean Theorem), or the magnitude of an applied Force vector, you have been using a Norm -- most likely the Euclidean Norm or Euclidean Distance. For example, for a vector $\mathbf{x}$ with $n$ dimensions, the Euclidean Norm looks like this:
$$
||\mathbf{x}||_2 = \sqrt{ x_1^2 + x_2^2 + \cdots x_n^2 }
$$
If you have ever had to compute the total Force Magnitude given its x and y components (for example, in Statics class), you have used the Euclidean Norm to do so. In that context, it served to take multiple components of a Force aggregate them in such a way as to tell you something about the **total force** -- by analogy, we will do the same thing here with linear regression, where each weight is like a component and we can use the Euclidean Norm to compute the total weight.

While Euclidean Norms may be quite useful or familiar to Engineers, it turns out that they are a special case of a much wider *family* of Norms called [*p-norms*](https://en.wikipedia.org/wiki/Lp_space#The_p-norm_in_finite_dimensions), which are defined as:
$$
||\mathbf{x}||_p = \left(|x_1|^p + |x_2|^p+\cdots + |x_n|^p\right)^{1/p} = \left(\sum_{i=1}^n \left| x_i \right|^p \right)^{1/p}
$$

Specifically, the Euclidean Norm is called the L2-norm, or sometimes just the 2-norm. To see why this is, just set $p=2$ in the above, and note how it corresponds to the Euclidean Norm that we all know and love. So, by setting $p$ to a number between $0$ and $\infty$, we can modify what the *total weight* means, and setting $p=2$ is the setting which we are all most familiar with. To get a visual sense of how norms vary, see below, which visualizes a line of "circle" of radius 1, but where the length of the line is determined by the p-norm. You will see that when p=2 this corresponds to what we are familiar with, but when p goes up or down things change.

![illustrations of p-Norms](2D_unit_balls.png "p-Norms")

For today, we will just focus on the L2-Norm, however we will revist norms again next week where we will see how changing the one we are using can have positive or negative effects in certain circumstances. 

For today's purpose, we will use the L2-Norm to help us penalize having linear regression models with really large weights, by essentially putting a cost on the total weight of the weight vector, where the total is measured by the L2-Norm. That is:
$$
Loss = \sum_{n=1}^{N}||y-w\cdot x||^2 + \alpha \cdot ||w||^2
$$
Where $\alpha$ is the price we pay for including more weight in the linear model. If it reduces our error cost enough to offset the cost of the increased weight, then that may be worth it to us. Otherwise, we would err on the side of using less weight overall.

This Regularization (i.e., increasing $\alpha$) essentially allows you to trade off bias and variance.

In [ ]:
# Import Ridge Regression
from sklearn.linear_model import Ridge

# alpha determines how much of 
# a penalty the weights incur
alphas = [0, 1e-20, 1e-10, 1e-7, 1e-5, 1, 10, 100]

# For the below example, let's
# just consider a 15-degree polynomial
d=15
np.random.seed(100)

for a in alphas:
    plt.figure(figsize=(7, 7))
    polynomial_features = PolynomialFeatures(degree=d,
                                             include_bias=False)
    #linear_regression = LinearRegression() #<- Note difference with next line
    linear_regression = Ridge(alpha=a)
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    # Fit model
    pipeline.fit(X[:, np.newaxis], y)
    # Get Training Accuracy
    score = pipeline.score(X[:, np.newaxis],y)

    # Plot things
    X_plot = np.linspace(0, 1, 100)
    plt.plot(X_plot, pipeline.predict(X_plot[:, np.newaxis]), label="Model")
    plt.plot(X_plot, true_fun(X_plot), '--',label="True function")
    plt.scatter(X, y, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    
    plt.title("Degree {}, Alpha {}\nTrain score = {:.3f}".format(
        d, a, score))
    plt.show()

###  Can we evaluate this more rigorously?
Yes, we can use data we have not seen yet to get an estimate of the generalization error. One popular way to do this is through Cross-Validation:

In [ ]:
# Now let's split the data into training and test data:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.2, random_state=0)

Let's take a look at what the above has actually done.

In [ ]:
print('X_train\n',X_train,'\n')
print('X_test\n',X_test,'\n')
print('y_train\n',y_train,'\n')
print('y_test\n',y_test,'\n')

plt.figure(figsize=(8,8))
plt.plot(X_plot, true_fun(X_plot), '--',label="True function")
# plot the training and testing points in colors
plt.scatter(X_train,y_train, label="Training data")
plt.scatter(X_test,y_test, label="Testing data")
plt.legend(loc="best")
plt.show()

Key idea in cross validation is to test the model on data that was separate from the data you trained on.

In [ ]:
alphas = [0, 1e-20, 1e-10, 1e-7, 1e-5, 1,10]
d=15
for a in alphas:
    plt.figure(figsize=(5, 5))
    #plt.setp(ax, xticks=(), yticks=())
    polynomial_features = PolynomialFeatures(degree=d,
                                             include_bias=False)
    #linear_regression = LinearRegression()
    linear_regression = Ridge(alpha=a)
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    #pipeline.fit(X[:, np.newaxis], y)
    pipeline.fit(X_train[:, np.newaxis], y_train)
    # Evaluate the models using crossvalidation
    #scores = cross_validation.cross_val_score(pipeline,
    #    X[:, np.newaxis], y, scoring="mean_squared_error", cv=10)
    
    testing_score = pipeline.score(X_test[:, np.newaxis],y_test)
    training_score = pipeline.score(X_train[:, np.newaxis],y_train)

    X_plot = np.linspace(0, 1, 100)
    plt.plot(X_plot, pipeline.predict(X_plot[:, np.newaxis]), label="Model")
    plt.plot(X_plot, true_fun(X_plot), '--',label="True function")
    plt.scatter(X, y, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    
    plt.title("Degree {}, Alpha {}\nTest score = {:.3f}\nTraining score = {:.3f}".format(
        d, a, testing_score,training_score))
    plt.show()